rerank-english-v3.0

In [1]:
pip install llama-index-postprocessor-cohere-rerank

  Using cached llama_index_core-0.10.68.post1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl.metadata (6.9 kB)
Using cached llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
Using cached tokenizers-0.19.1-cp310-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.21
    Uninstalling llama-index-core-0.12.21:
      Successfully uninstalled llama-index-core-0.12.21
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-cloud-services 0.6.1 requires llama-index-core>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-embeddings-fastembed 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-embeddings-huggingface 0.5.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-llms-huggingface 0.4.2 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-readers-smart-pdf-loader 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
transformers 4.49.0 requires tokenizers<0.22,>=0.21, but you 

In [52]:
!pip install flashrank==0.2.10

In [1]:
from langchain.llms import Cohere
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA
from langchain.retrievers.document_compressors import FlashrankRerank

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
import os

from getpass import getpass
import nest_asyncio

from dotenv import load_dotenv

nest_asyncio.apply()

load_dotenv()

True

In [3]:
CO_API_KEY = os.environ['CO_API_KEY'] or getpass("Enter your Cohere API key: ")

In [4]:
def load_data(url):
    loader = WebBaseLoader(url)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap= 50)
    chunks = text_splitter.split_documents(data)

    for idx, chunk in enumerate(chunks):
        chunk.metadata['id'] = idx

    return chunks

In [ ]:
# Using 'nomic-ai/nomic-embed-text-v1.5' model from Hugging Face under its respective license.

def vector_store(chunks):
    embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs={'trust_remote_code': True})

    faiss_vector_store = FAISS.from_documents(chunks, embeddings)
    
    return faiss_vector_store

In [6]:
llm = Cohere(model='command-xlarge', temperature= 0.5, cohere_api_key= CO_API_KEY)

C:\Users\1090135\AppData\Local\Temp\ipykernel_19096\224259106.py:1: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import Cohere``.
  llm = Cohere(model='command-xlarge', temperature= 0.5, cohere_api_key= CO_API_KEY)


In [7]:
llm

Cohere(client=<cohere.client.Client object at 0x0000019832E3B100>, async_client=<cohere.client.AsyncClient object at 0x00000198334B74C0>, model='command-xlarge', temperature=0.5, cohere_api_key=SecretStr('**********'))

In [8]:
def reranker(query, llm, vectorstore):

    compressor = FlashrankRerank()
    compressor.model_rebuild()

    compression_retriver = ContextualCompressionRetriever(
        base_compressor= compressor,
        base_retriever= vectorstore.as_retriever(search_kwargs={"k": 20})
    )

    chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriver)

    response = chain.invoke(query)

    return {
        "query": query,
        "final_answer": response["result"],
        "retrieval_method": "Re-ranking with FlashRank"
    }


In [9]:
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"

data = load_data(url)

In [10]:
vector_store = vector_store(data)


C:\Users\1090135\AppData\Local\anaconda3\envs\lil_llama_index\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [11]:
query = "what is the role of artificial intelligence?"

In [12]:
respones = reranker(query, llm, vector_store)

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|███████████████████████████████████████████████| 98.7M/98.7M [00:09<00:00, 10.8MiB/s]
INFO:httpx:HTTP Request: POST https://api.cohere.ai/v1/generate "HTTP/1.1 200 OK"


In [13]:
respones

{'query': 'what is the role of artificial intelligence?',
 'final_answer': ' Artificial intelligence plays a role in many different fields, including medicine, astronomy, economics, and more. It has a variety of useful applications, such as logical reasoning, learning, problem-solving, perception, and decision-making. \n\nOne of the goals of artificial intelligence is to develop methods and software that allow machines to act in ways that maximize their chances of achieving defined goals, although machines may not always be able to achieve the same level of complexity as that of humans. ',
 'retrieval_method': 'Re-ranking with FlashRank'}